## **Mapping Fire Impacts - Raster-based**

#### Reflection exercises for each group:

1. At the top of the Colab notebook that you share with everyone, please include your responses to the following questions:
  - Choose at least 1 dataset to explore in more detail.
  - What is the projection for this dataset?
  - Where can you find more information on how the data were collected and how to interpret the metadata?
  - Think about what data type each variable is.
  - Is it vector or raster data? What properties exist for each dataset?
  - What resolution are your data?

2. At the top of the Colab notebook, write a short summary detailing the processing steps in the notebook and your results.
  - Although these topics may be far removed from your own interests, how could these steps and analyses help in your own work?

3. OPTIONAL - Expand your script by adding additional processing, analysis, or other data.

As you're working through your exercise, **add code chunks to further document your scripts. Add additional comments to the code itself to clarify complicated processes.**




---



#### Acknowledgement

The code and comments for this project were adpated from NEON in [this tutorial](https://www.neonscience.org/resources/learning-hub/tutorials/intro-aop-data-google-earth-engine-tutorial-series#toggle-36). Check it out (or any of their other awesome tutorials!) to learn more!

### Objectives

In this first section, we'll explore the Soaproot Saddle NEON site in California before and after a 2020 burn. We'll map the reflectance data, RGB data, and difference in normalized burn ratio (dNBR).

<div id="ds-objectives" markdown="1">

### Background

The **Creek Fire** was a large wildfire that started in September 2020 in the Sierra National Forest, California and became one of the largest fires of the [2020 California wildfire season](https://en.wikipedia.org/wiki/2020_California_wildfires). This fire had burned into NEON's [Soaproot Saddle (SOAP)](https://www.neonscience.org/field-sites/soap) site by mid-September, causing a [high intensity burn over much of the site](https://www.neonscience.org/impact/observatory-blog/domain-digest-no-4-fire-field-sites-and-recovery) - standing trees became charcoal spires, shrubs and their root systems burned completely, the thick litter layer was incinerated, and the soil severely burned.

The NEON Airborne Observation Platform (AOP) conducted aerial surveys over SOAP in 2019 and 2021, a year before and after the Creek Fire.

In our introduction to AOP data, we learned how to calculate NDVI. Another common vegetation index is NBR, the normalized burn ratio. NBR is commonly used to asses burn severity. NBR is calculated as: <br>
> NBR = (NIR – SWIR) / (NIR + SWIR)

<img src='https://drive.google.com/uc?export=view&id=1zUP9oDdeQdiyQGL-2Vh1aqz8QCy03XPR' width=400> Image credit: [NASA ARSET Training Materials](https://appliedsciences.nasa.gov/sites/default/files/ChangeDetection_S1.pdf)


Then, dNBR is the difference in NBR, which is calcualted as <br>
> dNBR = prefire NBR - postfire NBR

The USGS proposes the following categories based on dNBR:<br>
<img src='https://drive.google.com/uc?export=view&id=1firJsT-ON56_LYII9MErCw1UC6V8Y3QD' width=600>Image credit: [Earth Lab](https://www.earthdatascience.org/courses/earth-analytics/multispectral-remote-sensing-modis/normalized-burn-index-dNBR/)

In [ ]:
import ee
import geemap
import geemap.colormaps as cm
ee.Authenticate()
ee.Initialize(project='')
geemap.ee_initialize(project='')

In [ ]:
# Import the AOP surface directional reflectance (SDR)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')
# Import the AOP red-green-blue imagery (RGB)
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')
# Import the AOP canopy height model data (CHM)
aopCHM = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30015-001')
# Import the AOP digital elevation model data (DEM)
aopDEM = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30024-001')

# Set up visualization params
RGB_bands = ['B053', 'B035', 'B019'] # These are the band names for the red, green, blue bands in the SDR data
rgbVis = {'min': 0, 'max': 255, 'gamma': 0.8} # This sets a nice range of values for mapping the RGB data
sdrVis = {'min': 0, 'max': 1260, 'gamma': 0.8} # This sets a nice range of values for mapping the RGB data

In [ ]:
### Code and comments adpated from NEON at https://www.neonscience.org/resources/learning-hub/tutorials/intro-aop-data-google-earth-engine-tutorial-series#toggle-36
'''------------------------- Normalized Difference Burn Ratio ----------------------------
The normalized burn ratio (NBR) is a normalized difference index using the shortwave-infrared (SWIR) and near-infrared (NIR) portions of the electromagnetic spectrum.
dNBR can be used as a metric to map fire extent and burn severity when calculating the difference between pre and post fire conditions.
'''

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_sdr = aopSDR \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2019_rgb = aopRGB \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


SOAP_2021_rgb = aopRGB \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


## Create an ROI that talks only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# calculate NBR
# B097: B365:
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# calculate dNBR
dNBR = pre_nbr \
        .subtract(post_nbr)

# Mapping
vis_params = {'min':-1, 'max':1, 'palette':cm.get_palette('RdYlGn_r', n_class=20)}
m = geemap.Map()


# # add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# # add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

# Bringing in vector data (fire exploration)
Now, let's compare our map of dNBR with existing fire data. The MTBS (Monitoring Trends in Burn Severity) data product is already publicly available on GEE.

In the next code chunk, we'll inspect 1 feature (representing a fire polygon) from MTBS. We extract the feature of interested by filtering the MTBS FeatureCollection to our area of interest and the year 2020.

In [ ]:
MTBS = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1") \
              .filterBounds(SOAP_2021_sdr.geometry()) \
              .map(lambda feat: feat.set('system:time_start', feat.get('Ig_Date'))) \
              .filterDate('2020-01-01', '2021-01-01')
MTBS

Now, let's overlay the MTBS data product on our dNBR map.

In [ ]:
# Mapping
vis_params = {'min':-1, 'max':1, 'palette':cm.get_palette('RdYlGn_r', n_class=20)}
m = geemap.Map()


# # add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

# add MTBS layer
m.addLayer(MTBS)

m.centerObject(ROI)
m

Now that we've seen the boundary of the 2020 Creek Fire, let's use the canopy 2019 and 2021 height model (CHM) and the Creek Fire Boundary to see the change in canopy height post-fire at the SOAP site.

First, we'll show the distribution of changes in canopy height post-fire for the entire burn region.

Then, we'll use dNBR thresholds to estimate "high severity" vs "low severity" burn regions within the burned area, and plot the distribution of changes in canopy height by estimated severity. <br>
*Note: If you're interested in fire severity classifications, MTBS also has a burn severity raster product available on GEE!*

In [ ]:
# Read in the CHM for 2019 and 2021
SOAP_2019_chm = aopCHM \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


SOAP_2021_chm = aopCHM \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

# Convert rasterize the MTBS polygon
creek_img = MTBS.reduceToImage(properties=['BurnBndAc'], reducer=ee.Reducer.anyNonZero()).clip(ROI).rename('burned')


# Calculate the pixel-by-pixel difference in canopy height between 2019 and 2021 for only the area in the Creek fire burn region
chm_diff_burned = SOAP_2021_chm.subtract(SOAP_2019_chm) \
                    .updateMask(creek_img.eq(1)) \
                    .sample(region=ROI, scale=30)

# Calculate the pixel-by-pixel difference in canopy height between 2019 and 2021 for only the area outside the Creek fire burn region
chm_diff_unburned = SOAP_2021_chm.subtract(SOAP_2019_chm) \
                    .updateMask(creek_img.eq(0)) \
                    .sample(region=ROI, scale=30)

# Plot difference in CHM
import geemap.chart as chart

options = {
    "xlabel": "Change in canopy height (m)",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}
chart.feature_histogram(chm_diff_burned, 'CHM', title='Change in canopy height post-fire in the Creek Fire boundary', **options)

In [ ]:
chart.feature_histogram(chm_diff_unburned, 'CHM', title='Change in canopy height post-fire outside the Creek Fire boundary', **options)

## Additional Resources

* **End-to-End Google Earth Engine**: If you'd like to continue exploring the Earth Engine processes and applications, <a href="https://courses.spatialthoughts.com/end-to-end-gee.html#automatic-conversion-of-javascript-code-to-python" target="_blank"> SpatialThoughts Course - Ujaval Gandhi </a> has some nice examples you can follow.

